In [104]:
import pandas as pd

In [105]:
# Calificaciones
df_calificaciones_2019 = pd.read_csv("data/DATA-PRUEBA_CALIFICACIONES-2019.csv",dtype={'cod_plan': str,'cod_asignatura': str,'cod_alumno': str})
df_calificaciones_2020 = pd.read_csv("data/DATA-PRUEBA_CALIFICACIONES-2020.csv",dtype={'cod_plan': str,'cod_asignatura': str,'cod_alumno': str})

In [106]:
# Repitencias
df_repitencias_2019 = pd.read_csv("data/DATA-PRUEBA_REPITENCIAS-2019.csv",dtype={'cod_plan': str,'cod_asignatura': str,'cod_alumno': str,'num_rep': int})
df_repitencias_2020 = pd.read_csv("data/DATA-PRUEBA_REPITENCIAS-2020.csv",dtype={'cod_plan': str,'cod_asignatura': str,'cod_alumno': str,'num_rep': int})

In [107]:
# Unir los datasets por tipo
df_calificaciones = pd.concat([df_calificaciones_2019, df_calificaciones_2020], ignore_index=True)
df_repitencias = pd.concat([df_repitencias_2019, df_repitencias_2020], ignore_index=True)

In [108]:
df_calificaciones

,cod_semestre,cod_facultad,cod_escuela,cod_plan,cod_asignatura,cod_alumno,val_calific_final
0,20191,1,1,2004,MH0460,14010243,17
1,20191,1,1,2004,MH0461,14010243,15
2,20191,1,1,2004,MH0462,14010243,16
3,20191,1,1,2004,MH0463,14010243,17
4,20191,1,1,2004,MH0464,14010243,13
...,...,...,...,...,...,...,...
532790,20202,20,2,2018,202W0607,18200281,15
532791,20202,20,2,2018,202W0607,18200315,14
532792,20202,20,2,2018,202W0607,18200323,16
532793,20202,20,2,2018,202W0607,18200320,13


In [109]:
df_repitencias


,cod_facultad,cod_escuela,cod_semestre,cod_alumno,cod_plan,cod_asignatura,asignatura,num_rep
0,1,1,20191,15010264,2015,M15033M,PATOLOGÍA ESPECIAL,1
1,1,1,20191,15010264,2015,M15035M,MEDICINA INTERNA,1
2,1,1,20191,15010264,2015,M15032M,PROYECTO DE TESIS II,3
3,1,1,20191,17010260,2015,M15028M,FISIOPATOLOGÍA,1
4,1,1,20191,12010250,2015,M15006M,INGLÉS MÉDICO,3
...,...,...,...,...,...,...,...,...
91058,20,2,20202,19200315,2018,202W0301,ALGORÍTMICA I,1
91059,20,2,20202,19200315,2018,202W0307,ORGANIZACIÓN Y ADMINISTRACIÓN,1
91060,20,2,20202,19200320,2018,202W0301,ALGORÍTMICA I,1
91061,20,2,20202,19200324,2018,202W0301,ALGORÍTMICA I,1


In [110]:
def process_dataframes(df_calificaciones, df_repitencias):
    # Calcular el promedio de notas por alumno, semestre, facultad, escuela, plan y asignatura
    df_calificaciones_avg = df_calificaciones.groupby(
        ['cod_alumno', 'cod_semestre', 'cod_facultad', 'cod_escuela', 'cod_plan', 'cod_asignatura']
    )['val_calific_final'].mean().reset_index()

    for col in ['cod_alumno', 'cod_facultad', 'cod_escuela', 'cod_plan', 'cod_asignatura']:
        df_calificaciones_avg[col] = df_calificaciones_avg[col].astype(str)
        df_repitencias[col] = df_repitencias[col].astype(str)

    for col in ['cod_alumno', 'cod_facultad', 'cod_escuela', 'cod_plan', 'cod_asignatura']:
        df_calificaciones_avg[col] = df_calificaciones_avg[col].str.strip()
        df_repitencias[col] = df_repitencias[col].str.strip()

    # Hacer el merge entre las calificaciones promedio y repitencias
    df_merged = pd.merge(
        df_calificaciones_avg.drop(columns=['cod_semestre']),
        df_repitencias.drop(columns=['asignatura', 'cod_semestre']),  # Eliminar la columna asignatura
        on=['cod_alumno', 'cod_facultad', 'cod_escuela', 'cod_plan', 'cod_asignatura'],
        how='left'
    )

    # Rellenar valores faltantes en `num_rep` con 0
    df_merged['num_rep'] = df_merged['num_rep'].fillna(0)

    return df_merged

In [111]:
df_merged = process_dataframes(df_calificaciones, df_repitencias)
df_merged

,cod_alumno,cod_facultad,cod_escuela,cod_plan,cod_asignatura,val_calific_final,num_rep
0,00110574,20,0,2009,206008,3.0,4.0
1,00110574,20,1,2009,206008,17.0,3.0
2,00110574,20,1,2009,206008,17.0,3.0
3,00110574,20,1,2009,206008,17.0,5.0
4,00110878,20,0,2009,201205,9.0,0.0
...,...,...,...,...,...,...,...
567228,998866,15,5,2009,S00204,11.0,1.0
567229,998866,15,5,2009,S00204,11.0,1.0
567230,998866,15,5,2009,S00204,11.0,1.0
567231,998866,15,5,2009,S00210,12.0,1.0


In [112]:
df_merged.to_csv("df_merged.csv", index=False)